In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import datasets, layers, models
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score
import numpy as np
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import random
import os


2024-01-05 11:25:06.571801: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 11:25:08.760863: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 11:25:08.760963: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 11:25:09.082165: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 11:25:09.775919: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-05 11:25:09.777386: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
dataset_dir = '/workspaces/LungsXray-FP/data/raw/MULTIPLE'

In [4]:
image_size = (150, 150)
batch_size = 32
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)
train_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    dataset_dir,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

Found 5266 images belonging to 4 classes.


Found 1315 images belonging to 4 classes.


In [5]:
num_classes = len(train_generator.class_indices)
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),  # Agrega Dropout para reducir el sobreajuste
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

2024-01-05 11:25:20.682754: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-05 11:25:20.708741: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-05 11:25:20.714829: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.


In [6]:
# Puedes experimentar con diferentes tasas de aprendizaje
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
# También puedes agregar ReduceLROnPlateau
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

In [8]:
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[reduce_lr]  # Agrega ReduceLROnPlateau como callback
)

Epoch 1/10


2024-01-05 11:25:21.901212: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.
2024-01-05 11:25:21.903270: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 18939904 exceeds 10% of free system memory.


165/165 [==============================] - 233s 1s/step - loss: 0.7451 - accuracy: 0.6996 - val_loss: 0.4652 - val_accuracy: 0.8297 - lr: 0.0010
Epoch 2/10
165/165 [==============================] - 236s 1s/step - loss: 0.4433 - accuracy: 0.8293 - val_loss: 0.3818 - val_accuracy: 0.8540 - lr: 0.0010
Epoch 3/10
165/165 [==============================] - 229s 1s/step - loss: 0.3905 - accuracy: 0.8551 - val_loss: 0.3399 - val_accuracy: 0.8707 - lr: 0.0010
Epoch 4/10
165/165 [==============================] - 227s 1s/step - loss: 0.3605 - accuracy: 0.8697 - val_loss: 0.3433 - val_accuracy: 0.8654 - lr: 0.0010
Epoch 5/10
165/165 [==============================] - 226s 1s/step - loss: 0.3332 - accuracy: 0.8773 - val_loss: 0.3471 - val_accuracy: 0.8692 - lr: 0.0010
Epoch 6/10
165/165 [==============================] - 223s 1s/step - loss: 0.3279 - accuracy: 0.8787 - val_loss: 0.2922 - val_accuracy: 0.8928 - lr: 0.0010
Epoch 7/10
165/165 [==============================] - 224s 1s/step - loss: 

In [9]:
test_loss, test_acc = model.evaluate(validation_generator, verbose=2)
print('\nExactitud en el conjunto de validación:', test_acc)

42/42 - 29s - loss: 0.2877 - accuracy: 0.9027 - 29s/epoch - 686ms/step

Exactitud en el conjunto de validación: 0.9026616215705872


In [10]:
model.save('modelo_softmax_optimizado.keras')

In [12]:
model = load_model('modelo_softmax_optimizado.keras')  # Ajusta el nombre de tu modelo
# Preprocesamiento de datos para el conjunto de prueba
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',  # Ajusta si tu modelo espera etiquetas categóricas
    shuffle=False
)

Found 6581 images belonging to 4 classes.


In [13]:
# Obtener las predicciones del modelo para el conjunto de prueba
predictions = model.predict(test_generator)
# Convertir las predicciones a clases (si es necesario)
predicted_classes = np.argmax(predictions, axis=1)
# Obtener las etiquetas verdaderas
true_classes = test_generator.classes
# Calcular la precisión y el recall por clase
classification_rep = classification_report(true_classes, predicted_classes)
accuracy = accuracy_score(true_classes, predicted_classes)
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Informe de clasificación:")
print(classification_rep)
print("Exactitud general:")
print(accuracy)
print("Matriz de confusión:")
print(conf_matrix)

206/206 [==============================] - 125s 606ms/step
Informe de clasificación:
              precision    recall  f1-score   support

           0       0.91      0.96      0.94      1589
           1       0.85      0.96      0.90      1341
           2       0.97      0.94      0.95      2616
           3       0.96      0.81      0.88      1035

    accuracy                           0.93      6581
   macro avg       0.93      0.92      0.92      6581
weighted avg       0.93      0.93      0.93      6581

Exactitud general:
0.9285822823279137
Matriz de confusión:
[[1532   13   15   29]
 [  14 1286   41    0]
 [   7  156 2451    2]
 [ 122   54   17  842]]
